In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as f
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader,Dataset,random_split
import PIL
import random
if (torch.cuda.is_available()):
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
class Network(nn.Module):
    def __init__(self):   
        super().__init__()
        self.l1 = nn.Conv2d(3,32,3,padding = 1)
        self.n1 = nn.BatchNorm2d(32)
        self.l2 = nn.Conv2d(32,32,3,padding = 1)
        self.n2 = nn.BatchNorm2d(32)
        self.l3 = nn.Conv2d(32,64,3,padding = 1)
        self.n3 = nn.BatchNorm2d(64)
        self.l4 = nn.Conv2d(64,64,3,padding = 1)
        self.n4 = nn.BatchNorm2d(64)
        self.l5 = nn.Conv2d(64,128,3,padding = 1)
        self.n5 = nn.BatchNorm2d(128)
        self.l6 = nn.Conv2d(128,128,3,padding = 1)
        self.n6 = nn.BatchNorm2d(128)
        self.l7 = nn.Conv2d(128,128,3,padding = 1)
        self.n7 = nn.BatchNorm2d(128)
        self.l8 = nn.Conv2d(128,256,3,padding = 1)
        self.n8 = nn.BatchNorm2d(256)
        self.l9 = nn.Conv2d(256,256,3,padding = 1)
        self.n9 = nn.BatchNorm2d(256)
        self.l10 = nn.Linear(9216,1024)
        self.l11 = nn.Linear(1024,10)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p = 0.3)
        self.drop2d = nn.Dropout2d(p = 0.3)
        self.softmax = nn.Softmax()
        self.pool = nn.MaxPool2d(2,stride = 2)
    def forward(self,xt,flag = False):
      x = xt
      convert = torch.zeros((x.shape[0],32-x.shape[1],x.shape[2],x.shape[3]))
      convert = convert.to(device)
      xt = torch.cat([xt,convert],dim = 1)
      x = self.l1(x)
      x = self.n1(x)
      x = self.relu(x)
      if(flag):
        x = self.drop2d(x)
      x = self.l2(x)
      x = x+xt
      x = self.n2(x)
      x = self.relu(x)
      x = self.pool(x)
      xt = x
      x = self.l3(x)
      x = self.n3(x)
      x = self.relu(x)
      if(flag):
        x = self.drop2d(x)
      x = self.l4(x)
      convert = torch.zeros(xt.shape,device=device)
      xt = torch.cat([xt,convert],dim = 1)
      x = x + torch.reshape(xt,x.shape)
      x = self.n4(x)
      x = self.relu(x)
      x = self.pool(x)
      xt = x
      x = self.l5(x)
      x = self.n5(x)
      x = self.relu(x)
      if(flag):
        x = self.drop2d(x)
      x = self.l6(x)
      convert = torch.zeros(xt.shape,device=device)
      xt = torch.cat([xt,convert],dim = 1)
      x = x + torch.reshape(xt,x.shape)
      x = self.n6(x)
      x = self.relu(x)
      if(flag):
        x = self.drop2d(x)
      x = self.pool(x)
      xt = x
      x = self.l7(x)
      x = self.n7(x)
      x = self.relu(x)
      if(flag):
        x = self.drop2d(x)
      x = self.l8(x)
      convert = torch.zeros(xt.shape,device=device)
      xt = torch.cat([xt,convert],dim = 1)
      x = x + torch.reshape(xt,x.shape)
      x = self.n8(x)
      x = self.relu(x)
      x = self.pool(x)
      x = self.l9(x)
      x = self.n9(x)
      x = self.relu(x)
      x = torch.reshape(x,(x.shape[0],-1,1,1))
      x = torch.squeeze(x,dim = 2)
      x = torch.squeeze(x,dim = 2)
      if(flag):
        x = self.drop(x)
      x = self.l10(x)
      x = torch.sigmoid(x)
      x = self.l11(x)
      x = self.softmax(x)
      return x
model = Network()
model = model.to(device)

In [ ]:
transform = transforms.ToTensor()
class Soupdata(Dataset):
  def __init__(self):
    super().__init__()
    transform = transforms.ToTensor()
    self.data = torchvision.datasets.STL10(root = '/',split = 'train',download = True,transform = transform)
    self.n = len(self.data)
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,i):
    return self.data[i]

In [ ]:
batch_size = 128
unsoup = torchvision.datasets.STL10(root = '/',split = 'unlabeled',download = True,transform = transform)
unsouploader = DataLoader(dataset=unsoup,batch_size = batch_size,shuffle = True)
soup =  Soupdata()
train_data,test_data = random_split(soup,[len(soup)-500,500])
souptrain = DataLoader(dataset=train_data,batch_size = batch_size,shuffle = True)
souptest = torchvision.datasets.STL10(root = '/',split = 'test',download = True,transform = transform)
soupval = DataLoader(dataset=test_data,batch_size = len(test_data),shuffle = True)
souptest = DataLoader(dataset=souptest,batch_size = 500,shuffle = True)


Extracting /stl10_binary.tar.gz to /
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
cost = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)
def test(data):
    c = 0
    s = 0
    for x,y in (data):
        with torch.no_grad():
            x =x.to(device)
            y = y.to(device)
            yt = model(x)
            yt = torch.squeeze(yt)
            yt = torch.argmax(yt, dim= 1)
            c = (y == yt).sum()
            s = y.shape[0]
        break
    return (100*c/s).item()

In [ ]:
def teste(data):
    c = 0
    s = 0
    for x,y in (data):
        with torch.no_grad():
            x =x.to(device)
            y = y.to(device)
            yt = model(x)
            yt = torch.squeeze(yt)
            yt = torch.argmax(yt, dim= 1)
            c += (y == yt).sum()
            s += y.shape[0]
    return (100*c/s).item()

In [ ]:
epochs = 40
flag = False
maxacc = 0
for j in range(epochs):
    for i,(xt,yt) in enumerate(souptrain) :
        xt = xt.to(device)
        yt = yt.to(device)
        y_pred = model(xt,True)
        y_pred = torch.squeeze(y_pred)
        loss = cost(y_pred,yt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i%20==0):
          acc = test(soupval)
          testacc = test(souptrain)
          if (acc>maxacc):
            maxacc = acc
          print(f'epoch {j+1} step {i} loss {loss} test_accuracy {acc} train_accuracy {testacc}')
        else:
          print(f'epoch {j+1} step {i} loss {loss}')
    if (flag):
      break


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Streaming output truncated to the last 5000 lines.
epoch 14 step 22 loss 1.5121526718139648
epoch 14 step 23 loss 1.5168099403381348
epoch 14 step 24 loss 1.5245540142059326
epoch 14 step 25 loss 1.5278013944625854
epoch 14 step 26 loss 1.508071780204773
epoch 14 step 27 loss 1.534417748451233
epoch 14 step 28 loss 1.525316596031189
epoch 14 step 29 loss 1.527147650718689
epoch 14 step 30 loss 1.5192689895629883
epoch 14 step 31 loss 1.4978702068328857
epoch 14 step 32 loss 1.5379093885421753
epoch 14 step 33 loss 1.5327365398406982
epoch 14 step 34 loss 1.5273231267929077
epoch 14 step 35 loss 1.5319414138793945
epoch 14 step 36 loss 1.4994162321090698
epoch 14 step 37 loss 1.5206022262573242
epoch 14 step 38 loss 1.5178029537200928
epoch 14 step 39 loss 1.5363563299179077
epoch 14 step 40 loss 1.5479066371917725 test_accuracy 68.4000015258789 train_accuracy 96.09375
epoch 14 step 41 loss 1.5159251689910889
epoch 14 step 42 loss 1.5317970514297485
epoch 14 step 43 loss 1.5192819833755

In [ ]:
print(maxacc)

77.4000015258789


In [ ]:
print(teste(souptest))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


62.525001525878906


In [ ]:
print(teste(souptest))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


60.525001525878906


In [ ]:
threshold = 0.99
l = []
initial = 0
final = 100000
for i,(x,y) in enumerate(unsoup):
  if (i<=initial):
    continue
  elif (i>=final):
    break
  xt = torch.unsqueeze(x,dim = 0)
  xt = xt.to(device)
  with torch.no_grad():
    y_pred = model(xt)
    y_pred = torch.squeeze(y_pred)
    label = torch.argmax(y_pred)
  if (y_pred[label]>threshold):
    l.append((x,label))
    print(i,y_pred[label])

1 tensor(0.9988, device='cuda:0')
5 tensor(0.9989, device='cuda:0')
18 tensor(0.9935, device='cuda:0')
22 tensor(0.9992, device='cuda:0')
44 tensor(0.9947, device='cuda:0')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Streaming output truncated to the last 5000 lines.
73419 tensor(0.9997, device='cuda:0')
73423 tensor(0.9996, device='cuda:0')
73425 tensor(0.9905, device='cuda:0')
73426 tensor(0.9962, device='cuda:0')
73427 tensor(0.9992, device='cuda:0')
73430 tensor(0.9942, device='cuda:0')
73443 tensor(0.9965, device='cuda:0')
73456 tensor(0.9987, device='cuda:0')
73458 tensor(0.9991, device='cuda:0')
73471 tensor(0.9981, device='cuda:0')
73472 tensor(0.9997, device='cuda:0')
73483 tensor(0.9984, device='cuda:0')
73488 tensor(0.9989, device='cuda:0')
73491 tensor(0.9985, device='cuda:0')
73492 tensor(0.9986, device='cuda:0')
73512 tensor(0.9994, device='cuda:0')
73519 tensor(0.9979, device='cuda:0')
73526 tensor(0.9929, device='cuda:0')
73531 tensor(0.9983, device='cuda:0')
73533 tensor(0.9978, device='cuda:0')
73535 tensor(0.9955, device='cuda:0')
73540 tensor(0.9989, device='cuda:0')
73545 tensor(0.9997, device='cuda:0')
73551 tensor(0.9984, device='cuda:0')
73553 tensor(0.9949, device='cuda:0')

In [ ]:
print(label.dtype)

torch.int64


In [ ]:
print(len(l))

18714


In [ ]:
class NewSoup(Dataset):
  def __init__(self,l):
    super().__init__()
    self.l = l
  def __len__(self):
    return len(self.l)
  def __getitem__(self,i):
    return self.l[i][0],self.l[i][1].item()
newsoup = NewSoup(l)
del l

In [ ]:
dataset = torch.utils.data.ConcatDataset([soup,newsoup])
print(len(dataset),len(soup),len(newsoup))
souptrain = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True)

23714 5000 18714
